# Imports

In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [2]:
import time
import pandas as pd
from google.cloud import bigquery
import plaid

from sql.bq_table_schemas import BqTableSchemas
from utils.bq_utils import BqUtils

# from utils.plaid_utils import PlaidUtils
from utils.plaid_utils import PlaidUtils
from utils.plaid_accounts import PlaidAccounts
from utils.plaid_transactions import PlaidTransactions
from utils.plaid_investments import PlaidInvestments


# constants
PLAID_CLIENT_ID = "65975384ab670e001c0aaf0d"
# PLAID_SECRET="56e33c77237c8c9e45f5c066b8b2fa" #production
PLAID_SECRET = "9294dd5ca4a5c99d90da56640f40e5"  # sandbox
PLAID_ENV = "sandbox"
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]
PLAID_HOST = plaid.Environment.Sandbox
# PLAID_REDIRECT_URI="https://localhost:3000/"
PLAID_ACCESS_TOKENS = [
    "access-sandbox-1902b6d6-33f3-49b8-a3ce-3213a5fe9a49",
    "access-sandbox-12481aea-33b6-4101-9f49-4e07ebbc1431",
    "access-sandbox-f415798a-74af-4e7f-b504-dc075dc5889c",
]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
plaid_accounts = PlaidAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
bq_tables = BqTableSchemas()

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [3]:
access_tokens = plaid_client.get_access_tokens(products=["investments"])
access_token = access_tokens["access_token"][0]

In [4]:
# investments_json = plaid_investments.get_investments(access_token=access_token)

holdings_df, securities_df = plaid_investments.get_investments_final(access_token)

In [5]:
holdings_df

,account_id,cost_basis,institution_price,institution_price_date,institution_price_datetime,institution_value,currency_code,unofficial_currency_code,quantity,security_id,vested_quantity,vested_value
0,93pgvXrp6JuljR6X39ZZFmgZZL9gldU4KRz4l,1.00,1.00000,2021-05-25,None,0.01000,USD,None,0.010000,d6ePmbPxgWCWmMVv66q9iPV94n91vMtov5Are,1.0,1.0
1,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,1.50,2.11000,2021-05-25,None,2.11000,USD,None,1.000000,KDwjlXj1Rqt58dVvmzRguxJybmyQL8FgeWWAy,NaN,NaN
2,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,10.00,10.42000,2021-05-25,None,20.84000,USD,None,2.000000,NDVQrXQoqzt5v3bAe8qRt4A7mK7wvZCLEBBJk,NaN,NaN
3,93pgvXrp6JuljR6X39ZZFmgZZL9gldU4KRz4l,0.01,0.01100,2021-05-25,None,110.00000,USD,None,10000.000000,8E4L9XLl6MudjEpwPAAgivmdZRdBPJuvMPlPb,NaN,NaN
4,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,120.03,39358.09375,2021-05-25,None,115.57268,USD,None,0.002936,9EWp9Xpqk1ua6DyXQb89ikMARWA6eyUzAbPMg,NaN,NaN
5,93pgvXrp6JuljR6X39ZZFmgZZL9gldU4KRz4l,40.00,42.15000,2021-05-25,None,210.75000,USD,None,5.000000,abJamDazkgfvBkVGgnnLUWXoxnomp5up8llg4,7.0,66.0
6,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,16.00,20.00000,2021-05-25,None,430.00000,USD,None,21.500000,lngLy3Le7vflLnAzKqwZs19l6bl8P5H1jG9Zz,NaN,NaN
7,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,23.00,27.00000,2021-05-25,None,636.30900,USD,None,23.567000,JDdP7XPMklt5vwPmDN45t3KAoWAPmjtpaW7DP,NaN,NaN
8,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,15.00,13.73000,2021-05-25,None,1373.68650,USD,None,100.050000,nnmo8doZ4lfKNEDe3mPJipLGkaGw3jfPrpxoN,NaN,NaN
9,vGz7rQXzlvijzAa6on33TNrooxKraeFqwWaqW,22.00,24.50000,2021-05-25,None,1855.87500,USD,None,75.750000,MD9eKXeplrt5yKRlzLqXiavwb6wrdxUb3wdnM,NaN,NaN


In [6]:
securities_df

,close_price,close_price_date,update_datetime,cusip,institution_id,institution_security_id,is_cash_equivalent,isin,currency_code,unofficial_currency_code,market_identifier_code,name,option_contract,proxy_security_id,security_id,sedol,ticker_symbol,type
0,0.01100,None,None,None,None,None,False,None,USD,None,None,Nflx Feb 01'18 $355 Call,None,None,8E4L9XLl6MudjEpwPAAgivmdZRdBPJuvMPlPb,None,NFLX180201C00355000,derivative
1,39358.09375,2021-05-25,None,None,None,None,True,None,USD,None,None,Bitcoin,None,None,9EWp9Xpqk1ua6DyXQb89ikMARWA6eyUzAbPMg,None,CUR:BTC,cash
2,8.55000,2024-04-23,None,None,None,None,False,None,USD,None,None,DoubleLine Total Return Bond I,None,None,AE5rBXra1AuZLE34rkvvIyG8918m3wtRzElnJ,B5ND9B4,DBLTX,mutual fund
3,27.00000,None,None,None,None,None,False,None,USD,None,None,Matthews Pacific Tiger Fund Insti Class,None,PDRMBVM8r3t5BAAqaKnmFPolGqlg8dtEGxywZ,JDdP7XPMklt5vwPmDN45t3KAoWAPmjtpaW7DP,None,MIPTX,mutual fund
4,2.11000,None,None,None,None,None,False,None,USD,None,None,Achillion Pharmaceuticals Inc.,None,None,KDwjlXj1Rqt58dVvmzRguxJybmyQL8FgeWWAy,None,ACHN,equity
5,24.50000,None,None,None,None,None,False,None,USD,None,None,Cambiar International Equity Institutional,None,WDwEPnEmJjt5lzNzv4w5F1rBmpBZvAHondAEZ,MD9eKXeplrt5yKRlzLqXiavwb6wrdxUb3wdnM,None,CAMYX,mutual fund
6,10.42000,None,None,None,None,None,False,None,USD,None,None,DoubleLine Total Return Bond Fund,None,AE5rBXra1AuZLE34rkvvIyG8918m3wtRzElnJ,NDVQrXQoqzt5v3bAe8qRt4A7mK7wvZCLEBBJk,None,DBLTX,mutual fund
7,NaN,None,None,None,None,None,False,None,USD,None,XCBT,None,None,None,PDRMBVM8r3t5BA9NmJdKfPolGqlg8dtEGB37n,None,PZ8XP9000,None
8,18.23000,2024-04-23,None,None,None,None,False,None,USD,None,None,Matthews Pacific Tiger Investor,None,None,PDRMBVM8r3t5BAAqaKnmFPolGqlg8dtEGxywZ,2529279,MAPTX,mutual fund
9,25.20000,2022-08-05,None,None,None,None,False,None,USD,None,XADF,International Seaways Inc 8.50% NT REDEEM 30/06/2023 USD 25,None,None,WDwEPnEmJjt5lzNzv4w5F1rBmpBZvAHondAEZ,BFXHBH6,INSW4638179,fixed income


# Test Transactions Sync

In [ ]:
plaid_init.create_temp_cursors_bq_table(confirm=False)

latest_cursors_df = plaid_init.get_latest_cursors()

# print(latest_cursors_df["access_token"][0])
# transactions_df, removed_df = plaid_client.get_transactions(
#     access_token=latest_cursors_df["access_token"][0],
#     item_id=latest_cursors_df["item_id"][0],
#     next_cursor=latest_cursors_df["next_cursor"][0],
# )

In [ ]:
from plaid.model.transactions_sync_request import TransactionsSyncRequest

In [ ]:
transactions_df.groupby(["personal_finance_category_primary", "personal_finance_category_detailed"], as_index=False)[
    "transaction_id"
].nunique()

In [ ]:
# transactions_df["personal_detailed"] = transactions_df.apply(
#     lambda row: row["personal_finance_category_detailed"][len(row["personal_finance_category_primary"]) + 1 :], axis=1
# )

# transactions_df.groupby(
#     ["personal_finance_category_primary", "personal_finance_category_detailed", "personal_detailed"], as_index=False
# )["transaction_id"].nunique()

# Test Delete all Partitions

In [ ]:
removed_transaction_bq = bq_tables.plaid_removed_transactions_YYYYMMDD()
transaction_bq = bq_tables.plaid_transactions_YYYYMMDD()
accounts_bq = bq_tables.plaid_accounts()

In [ ]:
bq.concat_table_name("project", "data", "table")

In [ ]:
bq.delete_all_partitions(
    project_id=transaction_bq["project_id"],
    dataset_id=transaction_bq["dataset_id"],
    table_id=transaction_bq["table_id"],
    confirm=True,
)

In [ ]:
from bqt import bqt